In [1]:
from text2brick.models import GraphLegoWorldData
import numpy as np

In [2]:
table = np.array([[0, 1, 1, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 1, 1, 0]])

In [3]:
graph = GraphLegoWorldData(img=table)

[0, 1, 2, 3]


In [5]:
vars(graph.graph)

{'graph': {},
 '_node': {0: {'x': 1, 'y': 3, 'brick_validity': False},
  1: {'x': 0, 'y': 2, 'brick_validity': False},
  2: {'x': 2, 'y': 1, 'brick_validity': True},
  3: {'x': 1, 'y': 0, 'brick_validity': True}},
 '_adj': {0: {1: {}}, 1: {0: {}}, 2: {3: {}}, 3: {2: {}}},
 'nodes': NodeView((0, 1, 2, 3))}